In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/scareware/'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-03 15:54:13.299672: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 97 images belonging to 11 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(97, 11)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(67, 64, 64, 3)

In [6]:
X_test.shape

(30, 64, 64, 3)

In [7]:
y_train.shape

(67, 11)

In [8]:
y_test.shape

(30, 11)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 11 # Changed from 10 to 11 for families on scareware

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(30, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(17, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-03 15:54:16.377756: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-03 15:54:16.705052: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 15:54:16.705148: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-03 15:54:16.706373: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 15:54:16.725437: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-03 15:54:16.7276

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 30)        840       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 17)        4607      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 17)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 17)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3332)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(67, 11)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([ 9,  1,  9,  3,  7,  8, 10, 10,  9,  6,  3,  0,  2,  5,  2,  6,  2,
        9,  3,  4, 10,  9,  5,  6, 10, 10,  5,  4,  3,  5,  8,  7,  3,  6,
        2,  6,  7,  6,  1,  5,  8,  4,  6,  6,  8,  4,  8,  4,  5,  1,  1,
        1,  2,  8,  7,  9,  6,  9,  4,  5,  6,  1,  0,  8, 10,  1,  5])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,  class_weight=class_weights)

2023-05-03 15:54:16.906304: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 1s 254ms/step - loss: 0.6276 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4765 - val_loss: 0.3358 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5924
Epoch 2/100
3/3 [==============================] - 0s 40ms/step - loss: 0.3789 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.5299 - val_loss: 0.3301 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.6248
Epoch 3/100
3/3 [==============================] - 0s 39ms/step - loss: 0.4018 - f1_m: 0.0167 - precision_m: 0.0972 - recall_m: 0.0091 - auc: 0.4777 - val_loss: 0.3136 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5844
Epoch 4/100
3/3 [==============================] - 0s 40ms/step - loss: 0.3728 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4908 - val_loss: 0.32

3/3 [==============================] - 0s 40ms/step - loss: 0.2524 - f1_m: 0.1137 - precision_m: 0.8333 - recall_m: 0.0612 - auc: 0.8428 - val_loss: 0.2727 - val_f1_m: 0.1818 - val_precision_m: 1.0000 - val_recall_m: 0.1000 - val_auc: 0.7281
Epoch 32/100
3/3 [==============================] - 0s 39ms/step - loss: 0.2579 - f1_m: 0.0505 - precision_m: 0.8333 - recall_m: 0.0260 - auc: 0.8068 - val_loss: 0.2717 - val_f1_m: 0.1818 - val_precision_m: 1.0000 - val_recall_m: 0.1000 - val_auc: 0.7085
Epoch 33/100
3/3 [==============================] - 0s 38ms/step - loss: 0.2615 - f1_m: 0.0500 - precision_m: 0.2917 - recall_m: 0.0273 - auc: 0.7635 - val_loss: 0.2689 - val_f1_m: 0.2353 - val_precision_m: 1.0000 - val_recall_m: 0.1333 - val_auc: 0.7149
Epoch 34/100
3/3 [==============================] - 0s 38ms/step - loss: 0.2600 - f1_m: 0.0671 - precision_m: 0.8333 - recall_m: 0.0352 - auc: 0.8075 - val_loss: 0.2664 - val_f1_m: 0.2353 - val_precision_m: 1.0000 - val_recall_m: 0.1333 - val_auc: 

3/3 [==============================] - 0s 37ms/step - loss: 0.1246 - f1_m: 0.8319 - precision_m: 0.9754 - recall_m: 0.7318 - auc: 0.9925 - val_loss: 0.2138 - val_f1_m: 0.5581 - val_precision_m: 0.9231 - val_recall_m: 0.4000 - val_auc: 0.8653
Epoch 64/100
3/3 [==============================] - 0s 37ms/step - loss: 0.1119 - f1_m: 0.7368 - precision_m: 0.8591 - recall_m: 0.6454 - auc: 0.9907 - val_loss: 0.2122 - val_f1_m: 0.5714 - val_precision_m: 1.0000 - val_recall_m: 0.4000 - val_auc: 0.8726
Epoch 65/100
3/3 [==============================] - 0s 38ms/step - loss: 0.1067 - f1_m: 0.8163 - precision_m: 0.9651 - recall_m: 0.7088 - auc: 0.9941 - val_loss: 0.2049 - val_f1_m: 0.5455 - val_precision_m: 0.8571 - val_recall_m: 0.4000 - val_auc: 0.8781
Epoch 66/100
3/3 [==============================] - 0s 39ms/step - loss: 0.1078 - f1_m: 0.8535 - precision_m: 1.0000 - recall_m: 0.7452 - auc: 0.9936 - val_loss: 0.1963 - val_f1_m: 0.6383 - val_precision_m: 0.8824 - val_recall_m: 0.5000 - val_auc: 

3/3 [==============================] - 0s 37ms/step - loss: 0.0501 - f1_m: 0.9696 - precision_m: 0.9825 - recall_m: 0.9570 - auc: 0.9987 - val_loss: 0.2359 - val_f1_m: 0.6296 - val_precision_m: 0.7083 - val_recall_m: 0.5667 - val_auc: 0.8757
Epoch 96/100
3/3 [==============================] - 0s 38ms/step - loss: 0.0362 - f1_m: 0.9868 - precision_m: 1.0000 - recall_m: 0.9740 - auc: 0.9999 - val_loss: 0.2403 - val_f1_m: 0.6154 - val_precision_m: 0.7273 - val_recall_m: 0.5333 - val_auc: 0.8744
Epoch 97/100
3/3 [==============================] - 0s 40ms/step - loss: 0.0389 - f1_m: 0.9868 - precision_m: 1.0000 - recall_m: 0.9740 - auc: 1.0000 - val_loss: 0.2452 - val_f1_m: 0.6275 - val_precision_m: 0.7619 - val_recall_m: 0.5333 - val_auc: 0.8724
Epoch 98/100
3/3 [==============================] - 0s 36ms/step - loss: 0.0398 - f1_m: 0.9831 - precision_m: 0.9831 - recall_m: 0.9831 - auc: 0.9999 - val_loss: 0.2556 - val_f1_m: 0.6275 - val_precision_m: 0.7619 - val_recall_m: 0.5333 - val_auc: 

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 21ms/step - loss: 0.2481 - f1_m: 0.6667 - precision_m: 0.8095 - recall_m: 0.5667 - auc: 0.8684


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 21ms/step - loss: 0.2481 - f1_m: 0.6667 - precision_m: 0.8095 - recall_m: 0.5667 - auc: 0.8684


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.666666567325592
